In [14]:
import ee
import geemap

from eelib import eefactory, eefuncs
from pipelines import plcfgs, factories

In [15]:
viewport = ee.FeatureCollection.from_file(
    filename = "../000-data/intersection_64.shp",
)

In [10]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [17]:
CFG = plcfgs.DataCubeCfg()
datacube = factories.DatacubeCollection(
    arg= 'projects/fpca-336015/assets/williston-cba'
)

geometry = datacube.geometry()

Map.centerObject(geometry, 6)
Map.addLayer(viewport, {}, 'Viewport')
Map.addLayer(geometry.dissolve(), {}, 'DC Geometry')

Map.addLayerControl()



In [25]:
# Sentinel - 1 Image Selection

targets = []

for season, cfg in CFG.seasons.items():
    dates = (cfg.get('start'), cfg.get('end'))
    base = ee.ImageCollection.s1Collection().\
        filter('relativeOrbitNumber_start == 64').\
        filterDate(*dates).\
        filterBounds(viewport)
    target_dates = [ee.Date(_) for _ in dates]
    
    # calc mid point
    mid_point = eefuncs.get_mid_point(target_dates)
    
    # difference the images date against the mid point
    target_collection = eefuncs.days_from_mid(
        collection=base,
        midPoint = mid_point
    )
    def setprops(element):
        return element.set('datestr', element.date().format('YYYY-MM-dd'),
                           'season', season)
    # inseart season 
    target_list = target_collection.filterDate(*dates)\
        .map(setprops).map(eefuncs.insert_groupid)\
        .toList(target_collection.size())
    target_list = target_list.map(lambda x: ee.Image(x))
    targets.append(target_list)
ee_list = ee.List(targets).flatten()


In [26]:
col = ee.ImageCollection(ee_list)
for season in CFG.seasons.keys():
    base = col.filter(f'season == "{season}"')
    
    dfms = base.aggregate_array('dfm').distinct().getInfo()
    print("Season: ")
    print("dfm, len")
    for dfm in dfms:
        query = base.filter(f'dfm == {dfm}')
        print(f"{dfm}, {query.size().getInfo()}")

Season: 
dfm, len
21, 3
9, 3
2, 3
14, 3
Season: 
dfm, len
28, 3
16, 3
4, 2
Season: 
dfm, len
15, 3


In [27]:
groupids = col.aggregate_array('groupid').distinct().getInfo()
groupids

['64_-122.36',
 '64_-122.83',
 '64_-123.37',
 '64_-122.47',
 '64_-122.94',
 '64_-123.33']